In [30]:
# Установка transformers
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [31]:
#Установка catboost
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [32]:
import pandas as pd
from itertools import groupby

In [33]:
#import необходимых модулей

import pandas as pd
import yaml
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, f1_score
from sklearn.utils.class_weight import compute_class_weight

import numpy as np
from datetime import datetime, timedelta

from catboost import CatBoostClassifier, Pool

In [34]:
#Считывание данных в DataFrame 
# Трейн со значениями равными и большими 7200
train= pd.read_csv('train_predobr_claster_vverh_7200.csv', sep=';', index_col=None)
# Тест по средним значениям (330 строк)
test_middle= pd.read_csv('test_predobr_middle7200_claster.csv', sep=';', index_col=None)
# таблица центров кластеров значений трейна выше 7200 в качестве групп классификации
tabliza= pd.read_csv('tabliza_clasters_middle.csv', sep=';', index_col=None)


In [35]:
train

,Номер кластера исходный,Расстояние до центра кластера,overall_worklogs,project_id,Id_razn,count_comments,id,assignee_id,creator_id,summary_to_eng,Year,Month,Day,Hour,DayOfWeek,Text_Summa,claster_centr_ishodn,mean_words_comment,Mesto_in_project
0,0,0.032940,7200,5,0,1,819949,93,93,"UI tests of the ""Profile"" section",2019,10,1,5,2,"UI tests of the ""Profile"" section Trusted and ...",11692,4.0,0.019231
1,0,0.019861,14400,5,0,0,819947,93,93,"UI tests of the section ""Personal account""",2019,10,1,6,2,"UI tests of the section ""Personal account""",11692,0.0,0.038462
2,0,0.028540,7800,5,475,2,819933,207,1,Make a division by mobile operators,2019,10,1,6,2,Make a division by mobile operators Added the ...,11692,16.0,0.153846
3,0,0.043181,17580,5,17,2,819924,264,207,Doesn't work with cdr_calc_durations,2019,10,1,8,2,Doesn't work with cdr_calc_durations [~mursidi...,11692,30.0,0.230769
4,10,0.056509,22500,5,0,0,819921,94,94,Fix crash addons (NegativeArraySizeException),2019,10,1,11,2,Fix crash addons (NegativeArraySizeException),30206,0.0,0.250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4914,0,0.001381,11880,48,0,0,688457,1,1,Investigate ANTMAN and find working CODs,2020,6,12,18,5,Investigate ANTMAN and find working CODs,11692,0.0,5.113941
4915,3,0.005558,67080,48,0,1,688422,1,1,Fix update script bug with JackfruitUsers,2020,6,12,20,5,Fix update script bug with JackfruitUsers Comm...,66322,62.0,5.194370
4916,0,0.019861,14400,48,0,1,688417,1,116,Teachers page returns 500 error,2020,6,15,9,1,Teachers page returns 500 error fixed. This is...,11692,11.0,5.207775
4917,10,0.030548,26040,48,0,0,688414,1,116,Make a test to check teachers pages for VN & TH,2020,6,15,9,1,Make a test to check teachers pages for VN & TH,30206,0.0,5.214477


In [36]:
test_middle

,id,Id_razn,mean_words_comment,count_comments,project_id,Mesto_in_project,Year,Month,Day,Hour,DayOfWeek,assignee_id,creator_id,summary_to_eng,Text_Summa
0,675961,180,41.500000,2,5,9.000000,2020,1,16,11,4,94,94,"During authorization, the user/driver does not...","During authorization, the user/driver does not..."
1,675955,3367,47.571429,7,5,9.076923,2020,1,17,5,5,1,54,The main problems for corp. department (as of ...,The main problems for corp. department (as of ...
2,675952,0,12.000000,1,5,9.096154,2020,1,17,7,5,71,54,Review of MPs on updating Map Box in the clien...,Review of MPs on updating Map Box in the clien...
3,675917,0,0.000000,0,5,9.538462,2020,2,1,2,6,112,1,"Drivers cannot click on the ""on the spot"" butt...","Drivers cannot click on the ""on the spot"" butt..."
4,675915,0,12.000000,1,5,9.557692,2020,2,1,2,6,112,1,The fare category in the fare card is not alwa...,The fare category in the fare card is not alwa...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325,688196,1,20.000000,2,48,5.690349,2020,7,2,18,4,1,1,Update Google Ads tag code (dynamic remarketin...,Update Google Ads tag code (dynamic remarketin...
326,688150,0,0.000000,0,48,5.764075,2020,7,9,3,4,82,82,Research about tests stage frozen and add fix,Research about tests stage frozen and add fix
327,688132,0,109.000000,1,48,5.790885,2020,7,9,10,4,1,1,No styles in one category,No styles in one category [~kirill] I have res...
328,702545,0,3.000000,1,48,9.323056,2020,9,24,11,4,1,1,Order to fix: audit result & save root cause,Order to fix: audit result & save root cause q...


In [37]:
tabliza

,claster_centr,Номер кластера
0,11692,0
1,1162850,1
2,7014000,2
3,66322,3
4,3393960,4
5,518312,5
6,262089,6
7,1698780,7
8,137018,8
9,2213040,9


In [38]:
best

,id,overall_worklogs
0,675975,17134
1,675972,17134
2,675965,17134
3,675961,12000
4,675955,12000
...,...,...
1065,702545,12000
1066,702528,17134
1067,702499,12000
1068,702376,2044


In [39]:
train.columns

Index(['Номер кластера исходный', 'Расстояние до центра кластера',
       'overall_worklogs', 'project_id', 'Id_razn', 'count_comments', 'id',
       'assignee_id', 'creator_id', 'summary_to_eng', 'Year', 'Month', 'Day',
       'Hour', 'DayOfWeek', 'Text_Summa', 'claster_centr_ishodn',
       'mean_words_comment', 'Mesto_in_project'],
      dtype='object')

In [40]:
# выбираем поля для нейросети СatBoost по бинарной классификации

X_train=train[['project_id', 'assignee_id', 'creator_id', 'summary_to_eng',
        'Year', 'Month', 'Day', 'Hour', 'DayOfWeek', 'Mesto_in_project',
       'Id_razn', 'mean_words_comment', 'count_comments', 'Text_Summa' ]]
y_train=train[['Номер кластера исходный']]

testnet_middle=test_middle[['project_id', 'assignee_id', 'creator_id', 'summary_to_eng',
        'Year', 'Month', 'Day', 'Hour', 'DayOfWeek','Mesto_in_project',
       'Id_razn', 'mean_words_comment', 'count_comments','Text_Summa' ]]

In [41]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4919 entries, 0 to 4918
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   project_id          4919 non-null   int64  
 1   assignee_id         4919 non-null   int64  
 2   creator_id          4919 non-null   int64  
 3   summary_to_eng      4919 non-null   object 
 4   Year                4919 non-null   int64  
 5   Month               4919 non-null   int64  
 6   Day                 4919 non-null   int64  
 7   Hour                4919 non-null   int64  
 8   DayOfWeek           4919 non-null   int64  
 9   Mesto_in_project    4919 non-null   float64
 10  Id_razn             4919 non-null   int64  
 11  mean_words_comment  4919 non-null   float64
 12  count_comments      4919 non-null   int64  
 13  Text_Summa          4919 non-null   object 
dtypes: float64(2), int64(10), object(2)
memory usage: 538.1+ KB


In [42]:
# Определяем веса для классов кластеров
classes = np.unique(y_train)
weights = compute_class_weight(class_weight='balanced', classes = np.unique(y_train), y = np.ravel(y_train))
class_weights = dict(zip(classes, weights))
class_weights

{0: 0.13759440559440558,
 1: 74.53030303030303,
 2: 447.1818181818182,
 3: 1.2704028925619835,
 4: 447.1818181818182,
 5: 29.812121212121212,
 6: 13.974431818181818,
 7: 223.5909090909091,
 8: 4.610121836925961,
 9: 223.5909090909091,
 10: 0.3851695247044084}

In [43]:
#Обучаем 

pool_train_solution = Pool(X_train, y_train, cat_features = ['project_id', 'assignee_id', 'creator_id','Year', 'Month', 'Day', 'Hour', 'DayOfWeek'], text_features = ['summary_to_eng', 'Text_Summa'])

pool_test_middle_solution = Pool(testnet_middle,cat_features = ['project_id', 'assignee_id', 'creator_id','Year', 'Month', 'Day', 'Hour', 'DayOfWeek'], text_features = ['summary_to_eng', 'Text_Summa'])


model_test = CatBoostClassifier(iterations=1200,  task_type='GPU',depth=12, verbose =True, random_seed=69,# learning_rate=0.015,
                               leaf_estimation_method= 'Gradient',bootstrap_type= 'Poisson',objective = 'MultiClass',eval_metric = 'Accuracy',loss_function='MultiClassOneVsAll', class_weights=class_weights)
model_test.fit(pool_train_solution)

0:	learn: 0.7141222	total: 571ms	remaining: 11m 24s
1:	learn: 0.7390186	total: 945ms	remaining: 9m 26s
2:	learn: 0.7631493	total: 1.3s	remaining: 8m 40s
3:	learn: 0.7741778	total: 1.69s	remaining: 8m 24s
4:	learn: 0.8044251	total: 2.18s	remaining: 8m 40s
5:	learn: 0.8094520	total: 2.58s	remaining: 8m 34s
6:	learn: 0.8156787	total: 2.98s	remaining: 8m 27s
7:	learn: 0.8188453	total: 3.35s	remaining: 8m 18s
8:	learn: 0.8172185	total: 3.72s	remaining: 8m 12s
9:	learn: 0.8278612	total: 4.13s	remaining: 8m 12s
10:	learn: 0.8373857	total: 4.5s	remaining: 8m 6s
11:	learn: 0.8366773	total: 4.89s	remaining: 8m 3s
12:	learn: 0.8425810	total: 5.26s	remaining: 8m
13:	learn: 0.8363607	total: 5.6s	remaining: 7m 54s
14:	learn: 0.8411999	total: 6.01s	remaining: 7m 55s
15:	learn: 0.8479120	total: 6.43s	remaining: 7m 55s
16:	learn: 0.8475837	total: 6.82s	remaining: 7m 54s
17:	learn: 0.8459212	total: 7.23s	remaining: 7m 54s
18:	learn: 0.8475760	total: 7.82s	remaining: 8m 6s
19:	learn: 0.8510132	total: 8.4

In [44]:
# предварительный тест схождения на всем обучающем множестве
y_pred = model_test.predict(X_train)
y_pred

array([[ 0],
       [ 0],
       [ 0],
       ...,
       [ 0],
       [10],
       [ 0]])

In [45]:
print("Значение метрики f1 на трейне по кластерам: ", f1_score(train['Номер кластера исходный'], y_pred, average='macro', zero_division = 0))

Значение метрики f1 на трейне по кластерам:  0.6311188296606048


In [46]:
train

,Номер кластера исходный,Расстояние до центра кластера,overall_worklogs,project_id,Id_razn,count_comments,id,assignee_id,creator_id,summary_to_eng,Year,Month,Day,Hour,DayOfWeek,Text_Summa,claster_centr_ishodn,mean_words_comment,Mesto_in_project
0,0,0.032940,7200,5,0,1,819949,93,93,"UI tests of the ""Profile"" section",2019,10,1,5,2,"UI tests of the ""Profile"" section Trusted and ...",11692,4.0,0.019231
1,0,0.019861,14400,5,0,0,819947,93,93,"UI tests of the section ""Personal account""",2019,10,1,6,2,"UI tests of the section ""Personal account""",11692,0.0,0.038462
2,0,0.028540,7800,5,475,2,819933,207,1,Make a division by mobile operators,2019,10,1,6,2,Make a division by mobile operators Added the ...,11692,16.0,0.153846
3,0,0.043181,17580,5,17,2,819924,264,207,Doesn't work with cdr_calc_durations,2019,10,1,8,2,Doesn't work with cdr_calc_durations [~mursidi...,11692,30.0,0.230769
4,10,0.056509,22500,5,0,0,819921,94,94,Fix crash addons (NegativeArraySizeException),2019,10,1,11,2,Fix crash addons (NegativeArraySizeException),30206,0.0,0.250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4914,0,0.001381,11880,48,0,0,688457,1,1,Investigate ANTMAN and find working CODs,2020,6,12,18,5,Investigate ANTMAN and find working CODs,11692,0.0,5.113941
4915,3,0.005558,67080,48,0,1,688422,1,1,Fix update script bug with JackfruitUsers,2020,6,12,20,5,Fix update script bug with JackfruitUsers Comm...,66322,62.0,5.194370
4916,0,0.019861,14400,48,0,1,688417,1,116,Teachers page returns 500 error,2020,6,15,9,1,Teachers page returns 500 error fixed. This is...,11692,11.0,5.207775
4917,10,0.030548,26040,48,0,0,688414,1,116,Make a test to check teachers pages for VN & TH,2020,6,15,9,1,Make a test to check teachers pages for VN & TH,30206,0.0,5.214477


In [47]:
# Проверим уровень схождения по r2 на трейне
train['Номер кластера']=y_pred
train=train.merge(tabliza, how="left", on="Номер кластера")
train['Номер кластера'].value_counts()

0     3680
10     917
3      237
8       41
6       28
5        8
9        3
1        3
4        1
7        1
Name: Номер кластера, dtype: int64

In [48]:
train['Номер кластера исходный'].value_counts()

0     3250
10    1161
3      352
8       97
6       32
5       15
1        6
9        2
7        2
4        1
2        1
Name: Номер кластера исходный, dtype: int64

In [49]:
train

,Номер кластера исходный,Расстояние до центра кластера,overall_worklogs,project_id,Id_razn,count_comments,id,assignee_id,creator_id,summary_to_eng,...,Month,Day,Hour,DayOfWeek,Text_Summa,claster_centr_ishodn,mean_words_comment,Mesto_in_project,Номер кластера,claster_centr
0,0,0.032940,7200,5,0,1,819949,93,93,"UI tests of the ""Profile"" section",...,10,1,5,2,"UI tests of the ""Profile"" section Trusted and ...",11692,4.0,0.019231,0,11692
1,0,0.019861,14400,5,0,0,819947,93,93,"UI tests of the section ""Personal account""",...,10,1,6,2,"UI tests of the section ""Personal account""",11692,0.0,0.038462,0,11692
2,0,0.028540,7800,5,475,2,819933,207,1,Make a division by mobile operators,...,10,1,6,2,Make a division by mobile operators Added the ...,11692,16.0,0.153846,0,11692
3,0,0.043181,17580,5,17,2,819924,264,207,Doesn't work with cdr_calc_durations,...,10,1,8,2,Doesn't work with cdr_calc_durations [~mursidi...,11692,30.0,0.230769,10,30206
4,10,0.056509,22500,5,0,0,819921,94,94,Fix crash addons (NegativeArraySizeException),...,10,1,11,2,Fix crash addons (NegativeArraySizeException),30206,0.0,0.250000,0,11692
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4914,0,0.001381,11880,48,0,0,688457,1,1,Investigate ANTMAN and find working CODs,...,6,12,18,5,Investigate ANTMAN and find working CODs,11692,0.0,5.113941,0,11692
4915,3,0.005558,67080,48,0,1,688422,1,1,Fix update script bug with JackfruitUsers,...,6,12,20,5,Fix update script bug with JackfruitUsers Comm...,66322,62.0,5.194370,0,11692
4916,0,0.019861,14400,48,0,1,688417,1,116,Teachers page returns 500 error,...,6,15,9,1,Teachers page returns 500 error fixed. This is...,11692,11.0,5.207775,0,11692
4917,10,0.030548,26040,48,0,0,688414,1,116,Make a test to check teachers pages for VN & TH,...,6,15,9,1,Make a test to check teachers pages for VN & TH,30206,0.0,5.214477,10,30206


In [50]:
print("Значение метрики r2 на трейне по секундам: ", r2_score(train['claster_centr'], train['overall_worklogs']))

Значение метрики r2 на трейне по секундам:  0.08535964984160194


In [51]:
#Получение ответов по средней части теста
y_pred_middle_solution = model_test.predict(pool_test_middle_solution)

In [52]:
# заводим результаты в тест для средней части
test_middle['Номер кластера']=y_pred_middle_solution
test_middle['Номер кластера'].value_counts()

0     298
10     26
3       6
Name: Номер кластера, dtype: int64

In [53]:
test_middle=test_middle.merge(tabliza, how="left", on="Номер кластера")
test_middle

,id,Id_razn,mean_words_comment,count_comments,project_id,Mesto_in_project,Year,Month,Day,Hour,DayOfWeek,assignee_id,creator_id,summary_to_eng,Text_Summa,Номер кластера,claster_centr
0,675961,180,41.500000,2,5,9.000000,2020,1,16,11,4,94,94,"During authorization, the user/driver does not...","During authorization, the user/driver does not...",0,11692
1,675955,3367,47.571429,7,5,9.076923,2020,1,17,5,5,1,54,The main problems for corp. department (as of ...,The main problems for corp. department (as of ...,0,11692
2,675952,0,12.000000,1,5,9.096154,2020,1,17,7,5,71,54,Review of MPs on updating Map Box in the clien...,Review of MPs on updating Map Box in the clien...,0,11692
3,675917,0,0.000000,0,5,9.538462,2020,2,1,2,6,112,1,"Drivers cannot click on the ""on the spot"" butt...","Drivers cannot click on the ""on the spot"" butt...",0,11692
4,675915,0,12.000000,1,5,9.557692,2020,2,1,2,6,112,1,The fare category in the fare card is not alwa...,The fare category in the fare card is not alwa...,0,11692
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325,688196,1,20.000000,2,48,5.690349,2020,7,2,18,4,1,1,Update Google Ads tag code (dynamic remarketin...,Update Google Ads tag code (dynamic remarketin...,0,11692
326,688150,0,0.000000,0,48,5.764075,2020,7,9,3,4,82,82,Research about tests stage frozen and add fix,Research about tests stage frozen and add fix,0,11692
327,688132,0,109.000000,1,48,5.790885,2020,7,9,10,4,1,1,No styles in one category,No styles in one category [~kirill] I have res...,0,11692
328,702545,0,3.000000,1,48,9.323056,2020,9,24,11,4,1,1,Order to fix: audit result & save root cause,Order to fix: audit result & save root cause q...,0,11692


In [54]:
test_middle_itog=test_middle[['id','claster_centr']]
test_middle_itog.columns=[['id','var2']]
test_middle_itog

,id,var2
0,675961,11692
1,675955,11692
2,675952,11692
3,675917,11692
4,675915,11692
...,...,...
325,688196,11692
326,688150,11692
327,688132,11692
328,702545,11692


In [55]:
test_middle_itog.to_csv('прогноз 11 кластер по средней части сид 69.csv', sep=',', index=None)